---

# **Chapter 12: Functional Programming Patterns**

## **Opening Context**

For decades, object‑oriented programming (OOP) has dominated the industry, with design patterns emerging to solve common problems of object creation, composition, and interaction. However, a parallel paradigm—**functional programming (FP)**—has steadily gained influence. FP emphasises **immutability**, **pure functions**, and **composition over inheritance**. Languages like Haskell, Scala, F#, and even multi‑paradigm languages (JavaScript, Python, C#, Java) have adopted functional features, making FP patterns increasingly relevant.

This chapter explores core functional patterns that every modern developer should know:

1. **Immutability and Pure Functions** – the foundational concepts that eliminate side effects and make code predictable.
2. **Monads, Functors, and Error Handling** – how types like `Maybe` (Option) and `Either` (Result) encapsulate optional values and errors, enabling safe, composable operations.
3. **Memoization** – caching function results to avoid redundant computation.
4. **Currying and Partial Application** – transforming functions to enable flexible reuse and point‑free style.

These patterns are not merely academic; they provide practical solutions to everyday problems: null safety, error handling, performance optimisation, and API design. By the end of this chapter, you will be able to recognise and apply functional patterns to write more robust, concise, and testable code.

---

## **12.1 Immutability and Pure Functions**

### **Intent**
*Ensure that data cannot be modified after creation (immutability) and that functions have no side effects and always return the same output for the same input (purity). These principles form the bedrock of functional programming.*

### **The Problem**

In traditional OOP, objects often have mutable state. This can lead to subtle bugs, especially in concurrent or asynchronous environments.

```typescript
// ❌ MUTABLE STATE: Unexpected changes
class BankAccount {
  constructor(public balance: number) {}
  
  withdraw(amount: number): void {
    if (this.balance >= amount) {
      this.balance -= amount; // Mutates internal state
    }
  }
}

const account = new BankAccount(100);
account.withdraw(50);
console.log(account.balance); // 50

// Somewhere else in the code, accidentally:
account.balance = 1000000; // Direct mutation breaks invariants
```

**Problems**:
- **Uncontrolled modification** – any code with a reference can change the object.
- **Reasoning difficulty** – to understand what the balance is, you must trace all code that might have modified it.
- **Concurrency hazards** – mutable shared state requires locks or complex synchronisation.

### **The Solution: Immutability**

Immutability means that once an object is created, it cannot be changed. Instead of modifying an object, you create a new one with the desired changes.

```typescript
// ✅ IMMUTABLE: Create new instances instead of mutating
class ImmutableBankAccount {
  // Readonly prevents reassignment, but deep immutability requires care
  constructor(public readonly balance: number) {}
  
  withdraw(amount: number): ImmutableBankAccount {
    if (this.balance >= amount) {
      return new ImmutableBankAccount(this.balance - amount);
    }
    return this; // No change
  }
}

const account = new ImmutableBankAccount(100);
const newAccount = account.withdraw(50);

console.log(account.balance);   // 100 (original unchanged)
console.log(newAccount.balance); // 50
```

**Deep Immutability**: For nested structures, you must ensure all levels are immutable. Libraries like **Immer** (JavaScript) or using `readonly` modifiers in TypeScript help.

```typescript
// Using readonly in TypeScript for nested objects
interface Address {
  readonly street: string;
  readonly city: string;
}

interface Person {
  readonly name: string;
  readonly address: Address;
}

// To "modify", create a copy
const person: Person = { name: "Alice", address: { street: "123 Main", city: "Springfield" } };
const updatedPerson: Person = {
  ...person,
  address: { ...person.address, street: "456 Oak" }
};
```

### **Pure Functions**

A **pure function** has two properties:
1. **Deterministic** – given the same inputs, it always returns the same output.
2. **No side effects** – it does not modify any external state, perform I/O, or depend on mutable external variables.

**Impure function examples**:

```typescript
// Impure: depends on external mutable variable
let taxRate = 0.2;
function calculateTax(amount: number): number {
  return amount * taxRate; // taxRate can change externally
}

// Impure: modifies input (side effect)
function addDiscount(prices: number[]): void {
  for (let i = 0; i < prices.length; i++) {
    prices[i] = prices[i] * 0.9; // Mutates the array
  }
}

// Impure: performs I/O
function logAndReturn(value: string): string {
  console.log(value); // Side effect
  return value;
}
```

**Pure function examples**:

```typescript
// Pure: always same output, no side effects
function add(a: number, b: number): number {
  return a + b;
}

// Pure: returns a new array, does not mutate input
function applyDiscount(prices: number[]): number[] {
  return prices.map(price => price * 0.9);
}

// Pure: no I/O, no external dependencies
function formatName(first: string, last: string): string {
  return `${first} ${last}`;
}
```

### **Benefits of Immutability and Purity**

1. **Predictability** – functions become easier to reason about and test.
2. **Referential transparency** – an expression can be replaced with its value without changing program behaviour (enables optimisations like memoisation).
3. **Thread safety** – immutable data can be shared across threads without locks.
4. **Easier debugging** – state changes are explicit; you can track snapshots.
5. **Composability** – pure functions are like building blocks that can be combined freely.

### **Trade‑offs**

- **Performance** – creating new objects can be more expensive than mutating existing ones. However, structural sharing (as in persistent data structures) mitigates this.
- **Memory usage** – more objects may be created. Again, persistent data structures and garbage collection help.
- **Interfacing with impure code** – boundaries with databases, file systems, or UI require handling side effects in a controlled way (e.g., using functional effect systems).

> **Key Insight**: Immutability and pure functions are not an all‑or‑nothing choice. Many codebases adopt a hybrid approach: use immutable data structures and pure functions in core logic, while allowing controlled mutability at the edges (I/O, UI).

---

## **12.2 Monads, Functors, and Error Handling (Maybe/Result types)**

### **The Problem: Handling Absence and Errors**

Null references (often called the "billion‑dollar mistake") plague software. A function may return `null` to indicate no value, leading to `NullPointerException` if not checked. Similarly, error handling with exceptions disrupts control flow and complicates composition.

```typescript
// ❌ NULL PROBLEM
function findUser(id: string): User | null {
  // ...
}

const user = findUser("123");
console.log(user.address.city); // Crash if user is null

// ❌ EXCEPTION PROBLEM
function divide(a: number, b: number): number {
  if (b === 0) throw new Error("Division by zero");
  return a / b;
}

try {
  const result = divide(10, 0);
} catch (e) {
  // Handle error
}
```

**Problems**:
- **Null checks** clutter code and are easy to forget.
- **Exceptions** break the function’s ability to be composed; they are not reflected in the return type, so callers may not expect them.

### **The Solution: Wrapper Types (Maybe/Option and Either/Result)**

Functional languages solve these problems by using wrapper types that explicitly encode the possibility of absence or failure.

#### **Functor**

A **Functor** is a type that implements a `map` method, which applies a function to the value inside the wrapper, returning a new wrapper of the same type.

```typescript
// Simplified Functor interface
interface Functor<T> {
  map<U>(fn: (value: T) => U): Functor<U>;
}
```

Arrays in JavaScript are functors: `[1,2,3].map(x => x*2)`.

#### **Maybe (Option) Monad**

`Maybe` (also called `Option`) represents a value that may or may not be present. It has two sub‑types: `Some` (contains a value) and `None` (empty).

```typescript
/**
 * MAYBE (OPTION) IMPLEMENTATION
 * A simple algebraic data type for optional values.
 */
type Maybe<T> = Some<T> | None<T>;

interface Some<T> {
  readonly tag: 'some';
  readonly value: T;
  map<U>(fn: (value: T) => U): Maybe<U>;
  flatMap<U>(fn: (value: T) => Maybe<U>): Maybe<U>;
  getOrElse(defaultValue: T): T;
}

interface None<T> {
  readonly tag: 'none';
  map<U>(fn: (value: T) => U): Maybe<U>;
  flatMap<U>(fn: (value: T) => Maybe<U>): Maybe<U>;
  getOrElse(defaultValue: T): T;
}

// Constructors
function some<T>(value: T): Some<T> {
  return {
    tag: 'some',
    value,
    map<U>(fn: (value: T) => U): Maybe<U> {
      return some(fn(this.value));
    },
    flatMap<U>(fn: (value: T) => Maybe<U>): Maybe<U> {
      return fn(this.value);
    },
    getOrElse(defaultValue: T): T {
      return this.value;
    }
  };
}

function none<T>(): None<T> {
  return {
    tag: 'none',
    map<U>(fn: (value: T) => U): Maybe<U> {
      return none();
    },
    flatMap<U>(fn: (value: T) => Maybe<U>): Maybe<U> {
      return none();
    },
    getOrElse(defaultValue: T): T {
      return defaultValue;
    }
  };
}

// Example usage
function findUser(id: string): Maybe<User> {
  // Simulate database lookup
  if (id === "123") {
    return some({ name: "Alice", address: { city: "Springfield" } });
  } else {
    return none();
  }
}

// Composing operations without null checks
const userCity = findUser("123")
  .map(user => user.address)
  .map(address => address.city)
  .getOrElse("Unknown city");

console.log(userCity); // "Springfield"

const missingCity = findUser("456")
  .map(user => user.address)
  .map(address => address.city)
  .getOrElse("Unknown city");

console.log(missingCity); // "Unknown city"
```

**Explanation**:
- `map` applies a function to the value if present; otherwise, it propagates the `None`.
- `flatMap` (also called `bind` or `>>=`) is used when the function itself returns a `Maybe`. This avoids nested `Maybe`s.
- `getOrElse` extracts the value or provides a default.

#### **Either (Result) Monad**

`Either` (often called `Result`) represents a value that can be one of two possibilities: a success (`Right` or `Ok`) or a failure (`Left` or `Error`). By convention, `Right` holds the success value, and `Left` holds an error.

```typescript
/**
 * EITHER (RESULT) IMPLEMENTATION
 * Represents a computation that may succeed (Right) or fail (Left).
 */
type Either<L, R> = Left<L, R> | Right<L, R>;

interface Left<L, R> {
  readonly tag: 'left';
  readonly value: L;
  map<U>(fn: (value: R) => U): Either<L, U>;
  flatMap<U>(fn: (value: R) => Either<L, U>): Either<L, U>;
  getOrElse(defaultValue: R): R;
}

interface Right<L, R> {
  readonly tag: 'right';
  readonly value: R;
  map<U>(fn: (value: R) => U): Either<L, U>;
  flatMap<U>(fn: (value: R) => Either<L, U>): Either<L, U>;
  getOrElse(defaultValue: R): R;
}

function left<L, R>(value: L): Left<L, R> {
  return {
    tag: 'left',
    value,
    map<U>(fn: (value: R) => U): Either<L, U> {
      return left(this.value) as any; // propagate error
    },
    flatMap<U>(fn: (value: R) => Either<L, U>): Either<L, U> {
      return left(this.value) as any;
    },
    getOrElse(defaultValue: R): R {
      return defaultValue;
    }
  };
}

function right<L, R>(value: R): Right<L, R> {
  return {
    tag: 'right',
    value,
    map<U>(fn: (value: R) => U): Either<L, U> {
      return right(fn(this.value));
    },
    flatMap<U>(fn: (value: R) => Either<L, U>): Either<L, U> {
      return fn(this.value);
    },
    getOrElse(defaultValue: R): R {
      return this.value;
    }
  };
}

// Example: safe division returning Either
function safeDivide(a: number, b: number): Either<string, number> {
  if (b === 0) {
    return left("Division by zero");
  } else {
    return right(a / b);
  }
}

// Chaining operations
const result = safeDivide(10, 2)
  .flatMap(x => safeDivide(x, 2))   // 10/2 =5, then 5/2 =2.5
  .map(x => x * 100)                // 250
  .getOrElse(0);

console.log(result); // 250

const errorResult = safeDivide(10, 0)
  .flatMap(x => safeDivide(x, 2))
  .map(x => x * 100)
  .getOrElse(0);

console.log(errorResult); // 0 (default)
```

**Explanation**:
- `map` only applies the function if the value is a `Right`; `Left` values are passed through unchanged.
- `flatMap` (often called `andThen` or `chain`) is used to sequence operations that themselves return an `Either`.
- This pattern allows error‑handling without exceptions; errors are just values that propagate.

### **Why Monads?**

A **monad** is a type that implements two operations:
- `return` (also called `pure` or `of`): wraps a value in the monad.
- `bind` (`flatMap` or `>>=`): sequences monadic operations.

Monads must also satisfy three laws (left identity, right identity, associativity) that ensure predictable behaviour. `Maybe` and `Either` are classic monads.

The power of monads lies in **composition**. Instead of writing nested `if` statements or `try/catch` blocks, you chain operations using `map` and `flatMap`. The monad handles the context (absence, failure) transparently.

### **Practical Libraries**

In real projects, you would not implement these from scratch. Use established libraries:

- **TypeScript/JavaScript**: `fp-ts` (functional programming in TypeScript) provides `Option`, `Either`, `Task`, etc.
- **Java**: `Optional` (for absence), `Either` from Vavr or custom `Result` types.
- **C#**: `Nullable<T>` for value types, `Option` from language‑ext, `Result` pattern.
- **Python**: `returns` library, or built‑in `Optional` with typing.

Example using `fp-ts`:

```typescript
import * as O from 'fp-ts/Option';
import * as E from 'fp-ts/Either';
import { pipe } from 'fp-ts/function';

const user = O.some({ name: 'Alice' });
const city = pipe(
  user,
  O.map(u => u.name),
  O.getOrElse(() => 'Unknown')
);
```

### **Error Handling with Result**

The `Result` pattern is increasingly adopted in modern languages (e.g., Rust's `Result<T, E>`, Swift's `Result`). It encourages explicit error handling without exceptions.

```typescript
// Rust-like Result in TypeScript (conceptual)
type Result<T, E> = { ok: true; value: T } | { ok: false; error: E };

function parseJSON(json: string): Result<any, SyntaxError> {
  try {
    return { ok: true, value: JSON.parse(json) };
  } catch (e) {
    return { ok: false, error: e };
  }
}

// Usage
const result = parseJSON('{"a":1}');
if (result.ok) {
  console.log(result.value.a);
} else {
  console.error('Parse failed:', result.error);
}
```

---

## **12.3 Memoization Pattern: Caching Function Results**

### **Intent**
*Cache the results of expensive function calls and return the cached result when the same inputs occur again.*

### **The Problem**

Some computations are expensive (e.g., recursive calculations, database queries, complex transformations). Without caching, the same work may be repeated unnecessarily.

```typescript
// Expensive recursive Fibonacci (exponential time)
function fib(n: number): number {
  if (n <= 1) return n;
  return fib(n - 1) + fib(n - 2);
}

console.time('fib(40)');
fib(40); // Takes ~1 second
console.timeEnd('fib(40)');
```

Calling `fib(40)` multiple times repeats the entire calculation.

### **The Solution: Memoization**

Memoization stores results in a cache keyed by the function's arguments. When the function is called again with the same arguments, the cached result is returned.

```typescript
// Memoization higher‑order function
function memoize<T extends (...args: any[]) => any>(fn: T): T {
  const cache = new Map<string, ReturnType<T>>();
  
  return function(...args: Parameters<T>): ReturnType<T> {
    const key = JSON.stringify(args); // Simple key; for complex args, use a better strategy
    if (cache.has(key)) {
      console.log('Cache hit for', args);
      return cache.get(key)!;
    }
    console.log('Cache miss for', args);
    const result = fn(...args);
    cache.set(key, result);
    return result;
  } as T;
}

// Apply memoization to fib
const memoizedFib = memoize(fib);

console.time('memoizedFib(40)');
memoizedFib(40); // Computes and caches
console.timeEnd('memoizedFib(40)');

console.time('memoizedFib(40) again');
memoizedFib(40); // Returns instantly from cache
console.timeEnd('memoizedFib(40) again');
```

**Explanation**:
- `memoize` wraps the original function and maintains a `Map` of argument strings to results.
- On each call, it computes a key (here using `JSON.stringify` – adequate for primitive arguments, but for objects you might need a custom key generator or use `WeakMap` for object keys).
- If the key exists, the cached value is returned; otherwise, the original function is called and the result cached.

### **Memoization for Recursive Functions**

When memoizing a recursive function, ensure the memoized version calls itself recursively. In the example above, `memoizedFib` calls the original `fib` (which is not memoized). To fully memoize, the recursive function should call the memoized version.

```typescript
// Properly memoized recursive function
const fibMemo = memoize(function fib(n: number): number {
  if (n <= 1) return n;
  return fibMemo(n - 1) + fibMemo(n - 2); // Calls memoized version
});

console.time('fibMemo(40)');
fibMemo(40);
console.timeEnd('fibMemo(40)'); // Much faster than non‑memoized
```

### **Memoization with Multiple Arguments**

`JSON.stringify` works for multiple arguments as they become part of the array.

```typescript
const expensiveSum = memoize((a: number, b: number, c: number) => {
  console.log('Computing sum...');
  return a + b + c;
});

expensiveSum(1, 2, 3); // Computes
expensiveSum(1, 2, 3); // Cache hit
```

### **Memoization Considerations**

1. **Cache size**: Unbounded caches can lead to memory leaks. Use a bounded cache (e.g., LRU cache) for production.
2. **Key generation**: For objects, using `JSON.stringify` may be slow and can fail with circular references. Consider using a library like `lru-cache` that handles object keys.
3. **Pure functions only**: Memoization only works for pure functions; if the function depends on external state or has side effects, caching can produce incorrect results.
4. **Thread safety**: In concurrent environments, the cache must be thread‑safe.

### **Real‑World Use Cases**

- **React components**: `React.memo` and `useMemo` are built‑in memoization for components and values.
- **Redux selectors**: `reselect` creates memoized selectors.
- **Database query results**: Caching query results for identical parameters.
- **Expensive computations**: Image processing, data transformations.

---

## **12.4 Currying and Partial Application**

### **Intent**
*Transform a function that takes multiple arguments into a sequence of functions, each taking a single argument (currying), or fix a subset of arguments to produce a new function (partial application).*

### **The Problem**

Sometimes you want to reuse a function with some arguments preset, or you want to build up a function incrementally.

```typescript
// A function that takes three arguments
function formatMessage(prefix: string, message: string, suffix: string): string {
  return `${prefix} ${message} ${suffix}`;
}

// You often call it with the same prefix and suffix:
console.log(formatMessage(">>>", "Hello", "<<<"));
console.log(formatMessage(">>>", "World", "<<<"));
```

Every call repeats the same prefix and suffix.

### **The Solution: Partial Application**

**Partial application** fixes a number of arguments to a function, producing a new function that takes the remaining arguments.

```typescript
// Generic partial application helper
function partial<T extends any[], U extends any[], R>(
  fn: (...args: [...T, ...U]) => R,
  ...preset: T
): (...args: U) => R {
  return function(...later: U): R {
    return fn(...preset, ...later);
  };
}

const formatWithMarkers = partial(formatMessage, ">>>", "<<<");
console.log(formatWithMarkers("Hello")); // >>> Hello <<<
console.log(formatWithMarkers("World")); // >>> World <<<
```

**Explanation**:
- `partial` takes a function and some arguments (`preset`). It returns a new function that, when called with the remaining arguments (`later`), calls the original with all arguments combined.

### **Currying**

**Currying** transforms a function that takes multiple arguments into a chain of functions that each take a single argument. After currying, calling the function with fewer arguments returns a new function waiting for the rest.

```typescript
// Currying helper
function curry<T extends any[], R>(
  fn: (...args: T) => R
): T extends [] ? () => R :
   T extends [infer A] ? (a: A) => R :
   T extends [infer A, ...infer Rest] ? (a: A) => ReturnType<typeof curry<Rest, R>> :
   never;

function curry<T extends any[], R>(fn: (...args: T) => R): any {
  return function curried(...args: any[]): any {
    if (args.length >= fn.length) {
      return fn(...args as T);
    } else {
      return (...more: any[]) => curried(...args, ...more);
    }
  };
}

// Curried formatMessage
const curriedFormat = curry(formatMessage);
const withPrefix = curriedFormat(">>>");          // (message: string, suffix: string) => string
const withPrefixAndSuffix = withPrefix("<<<");    // (message: string) => string
console.log(withPrefixAndSuffix("Hello"));         // >>> Hello <<<
console.log(withPrefixAndSuffix("World"));         // >>> World <<<
```

**Explanation**:
- `curry` returns a function that collects arguments until it has enough to call the original.
- When called with fewer arguments than the original function's arity, it returns a new function that expects the rest.

### **Difference Between Currying and Partial Application**

- **Partial application** produces a function with some arguments fixed; the resulting function may take multiple arguments.
- **Currying** produces a chain of unary (single‑argument) functions. You can think of currying as a specific form of partial application where each step takes exactly one argument.

In practice, many libraries (like Lodash/fp) provide both.

### **Practical Benefits**

1. **Function composition**: Curried functions work beautifully with `compose` and `pipe`.
2. **Code reuse**: Create specialized functions from general ones.
3. **Readability**: Point‑free style can sometimes make code more declarative.

Example using `pipe` with curried functions:

```typescript
// Assume we have curried map, filter, reduce
const map = <T, U>(fn: (x: T) => U) => (arr: T[]) => arr.map(fn);
const filter = <T>(pred: (x: T) => boolean) => (arr: T[]) => arr.filter(pred);
const reduce = <T, U>(fn: (acc: U, x: T) => U, init: U) => (arr: T[]) => arr.reduce(fn, init);

const sumOfEvens = pipe(
  filter((x: number) => x % 2 === 0),
  map((x: number) => x * 2),
  reduce((acc, x) => acc + x, 0)
);

console.log(sumOfEvens([1,2,3,4,5])); // (2*2)+(4*2)=4+8=12
```

### **Currying in Languages**

- **JavaScript**: Not natively curried, but libraries like Lodash (`_.curry`) provide it.
- **Haskell**: All functions are curried by default.
- **Scala**: Methods can be curried using multiple parameter lists.
- **Python**: `functools.partial` provides partial application; currying can be implemented manually.

---

## **Chapter Summary**

Functional programming patterns offer a different lens for structuring code, emphasising immutability, composition, and explicit handling of effects.

1. **Immutability and Pure Functions** – By avoiding mutation and side effects, code becomes more predictable, testable, and concurrency‑friendly. Immutable data structures and pure functions form the foundation upon which other functional patterns are built.

2. **Monads, Functors, and Error Handling** – Types like `Maybe` (Option) and `Either` (Result) encapsulate optionality and errors, enabling safe composition without null checks or exceptions. They are functors (support `map`) and monads (support `flatMap`), allowing you to chain operations while handling absence or failure automatically.

3. **Memoization** – Caching function results based on inputs avoids redundant computation. This is a practical optimisation that relies on function purity; it can be applied selectively to improve performance.

4. **Currying and Partial Application** – These techniques transform functions to allow incremental argument supply, enabling powerful composition and code reuse. They are especially useful in combination with `pipe`/`compose` to create data pipelines.

**Key Insight**: Functional patterns are not a replacement for OOP patterns but rather a complementary set of tools. Many modern languages allow mixing paradigms, letting you choose the best approach for each problem. Mastering these patterns equips you to write code that is both robust and expressive.

---

## **Next Chapter Preview**

**Chapter 13: Layered and Modular Architecture (N-Tier, Hexagonal, Onion)**

We now return to architectural patterns, exploring how to organise entire applications. Chapter 13 will cover **N‑Tier Architecture** (traditional layering), **Hexagonal Architecture** (Ports and Adapters) that isolates the core domain, and **Onion Architecture** that emphasises dependency inversion. These patterns provide blueprints for building maintainable, testable, and adaptable systems at scale.



<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='11. patterns_in_modern_language_features.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../4. architectural_patterns/13. layered_and_modular_architecture.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
